In [1]:
from dataset import CocoDataset
from torch.utils.data import DataLoader

In [2]:
ann_file = "/home/geng/fiftyone/coco-2017/validation/labels.json"
dataset_type = 'CocoDataset'  # 数据集类型，这将被用来定义数据集。
data_root = '/home/geng/fiftyone/coco-2017/validation/data/'  # 数据的根路径。

img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
input_size = (1216,800)

test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=input_size,
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=False),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
dataset = CocoDataset(ann_file=ann_file, pipeline=test_pipeline,data_root=data_root)

loading annotations into memory...
Done (t=0.76s)
creating index...
index created!


In [4]:
from mmcv.parallel import collate
batch_size = 1
data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=collate
        )

In [5]:
from tqdm import tqdm
import onnxruntime
import torch
import numpy as np
#  onnx推理过程
def onnxruntime_inference(data_loader,onnxruntime_model_path):
    sess = onnxruntime.InferenceSession(path_or_bytes=onnxruntime_model_path, providers=['CUDAExecutionProvider'])
    input_placeholder_name = sess.get_inputs()[0].name
    outputnames = [x.name for x in sess.get_outputs()]
    outputs = []
    for x in tqdm(data_loader):
        input_tensor = x["img"][0]
        img_metas = x["img_metas"][0].data[0][0]
        output = sess.run(input_feed={input_placeholder_name: input_tensor.cpu().numpy()},output_names=outputnames)
        img_metas["output"] = output
        outputs.append(img_metas)
    return outputs

In [6]:
"""
将任意模型的输出转换为常规输出
[N,result]  N为样本数
result: [C,[M,5]] 其中C为类别,M为bbox数目，5为左上角、右下角坐标与置信分数。
并且根据尺度进行复原
"""
# 模型后处理过程
def post_process(model_name,outputs):
    if model_name == "retinanet":
        results = []
        for output in outputs:
            img_scale = output["scale_factor"] 
            bboxs,labels = output["output"]
            result = [[] for _ in range(len(dataset.cat_ids))]
            bboxs,labels = bboxs[0],labels[0]
            for i,label in enumerate(labels):
                bboxs[i][:4] = bboxs[i][:4] / img_scale  # 进行bbox的尺寸复原
                result[label].append(bboxs[i])
            results.append(result)
        return results

In [11]:
onnxruntime_model_path = "/home/geng/tinyml/ppq/benchmark/detection/FP32_model/Retinanet-FP32-end.onnx"
outputs = onnxruntime_inference(data_loader,onnxruntime_model_path)
results = post_process("retinanet",outputs)

  0%|          | 0/4952 [00:00<?, ?it/s]


In [17]:
dataset.results2json(results=results,outfile_prefix="retinanet")  # 结果统一保存为coco json

{'bbox': 'retinanet.bbox.json', 'proposal': 'retinanet.bbox.json'}

In [3]:
result_path = "/home/geng/tinyml/ppq/benchmark/detection/result/retinanet.bbox.json"
dataset.evaluate(results_json_path=result_path)  
# dataset.evaluate(results=results) # 直接使用输出结果进行测试  


Evaluating bbox...
Loading and preparing results...
DONE (t=1.62s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=83.80s).
Accumulating evaluation results...
DONE (t=21.69s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.524
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.362
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.366
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.450
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.520
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.520
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.520
 Average Recall     (AR) @[ IoU=0.50:0.

UnboundLocalError: local variable 'tmp_dir' referenced before assignment